# Import from ELN

Importing data from an electronic laboratory notebook

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from aiida import load_profile

load_profile();

In [ ]:
import json
import os
import urllib.parse as urlparse

import traitlets as tr
from pyld import jsonld

import aiidalab_widgets_base as awb

In [ ]:
# sys.path.append(os.path.dirname("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/"))
# from materialMLinfo import Molecule, slots
# def extract_molecule_data_with_linkml(molecule_jsonld):
#     expanded = jsonld.expand(molecule_jsonld)
#     molecule_data = {}
#     for obj in jsonld_object:
#         if '@type' in obj and str(Molecule.class_class_uri) in obj['@type']:
#             molecule_data['name'] = obj.get(str(slots.name.uri), [None])[0]['@value']
#             molecule_data['smiles'] = obj.get(str(slots.smiles.uri), [None])[0]['@value']
#     return molecule_data

In [ ]:
# Extract data from the converted JSON-LD
def extract_molecule_data(jsonld_object, context):  # noqa: ARG001
    expanded = jsonld.expand(molecule_jsonld)
    compacted = jsonld.compact(expanded, context)
    return {"name": compacted["name"], "smiles": compacted["smiles"]}


def read_json(filepath: str) -> dict:
    return json.load(open(filepath))


url = urlparse.urlsplit(jupyter_notebook_url)  # noqa: F821
parsed_url = urlparse.parse_qs(url.query)
params = {key: value[0] for key, value in parsed_url.items()}
molecule_jsonld = json.loads(params["molecule_info"])

molecule_info_valid = False

jsonld_context_filename = os.path.join(
    os.sep,
    "home",
    "jovyan",
    "aiida-openbis",
    "Notebooks",
    "Metadata_Schemas_LinkML",
    "materialMLinfoContext.jsonld",
)


aiida_context = read_json(jsonld_context_filename)

try:
    molecule_data = extract_molecule_data(molecule_jsonld, aiida_context)
    params["molecule_info"] = json.dumps(molecule_data)
    eln_widget = awb.ElnImportWidget(path_to_root="../../", **params)
    molecule_info_valid = True
except ValueError as e:
    print(e)

In [ ]:
if molecule_info_valid:
    object_displayed = awb.AiidaNodeViewWidget()
    open_in_app = awb.OpenAiidaNodeInAppWidget(path_to_root="../../")

    _ = tr.dlink((eln_widget, "node"), (object_displayed, "node"))
    _ = tr.dlink((eln_widget, "node"), (open_in_app, "node"))

## Selected object:

In [ ]:
if molecule_info_valid:
    display(object_displayed)
    display(eln_widget)

## What's next?

In [ ]:
if molecule_info_valid:
    display(open_in_app)